# <span style="color:#F72585"><center>Análisis exploratorio de Modelos Bayesianos</center></span>


<center>(TFP)<center>

## <span style="color:#4361EE">Introducción</span>


Se introducen los elementos esenciales del análisis exploratorio de  Modelos Bayesianos. Usaremos la librería [Arviz](https://arviz-devs.github.io/arviz/) que es una libería desarrollada para hacer estos análisis en Python. 

En particular, Arviz puede trabajar con las salidas de TFP.

En este cuaderno usamos el ejemplo de 8 escuelas (eight schools) que utilizamos en el [primer ejemplo de Stan en este curso](Stan_Ejemplo_1_eight_schools.ipynb)

## <span style="color:#4361EE">El modelo estadístico para ete tutorial</span>


Supongamos que tenemos observaciones $[y_n |x_n], n=1,\ldots,N$, y asumamos el modelo Bayesiano. 

**Modelo con efectos aleatorios**

- Estima los hiperparámetros $\mu$ and $\tau$
- Predice los efectos aleatorios $\eta_i$


$$
\begin{align}
y_i &\sim \mathcal{N}(\theta_i,\sigma_i^2), \text{ known } \sigma_i^2\\
\theta_i & = \mu + \tau \times \eta_i \\
\eta_i &\sim \mathcal{N}(0, 1)\\
\mu &\sim \mathcal{N}(0, 5)\\
\tau &\sim \mathcal{Cauchy}(0, 5)1_{\tau>0.0}
\end{align}
$$


### <span style="color:#4CC9F0">Los datos</span>


Los datos corresponden a la estimación (estandarizada) de un puntaje realizada en las ocho escuelas observadas.


```{table}
|School |Estim. Treatment Effect | Estim. Stand. Error |
|---|---|---|
|A |28| 15|
|B |8 |10|
|C| -3| 16|
|D| 7| 11|
|E| -1 |9|
|F| 1| 11|
|G |18| 10|
|H| 12| 18|
```

In [2]:
# Importamos las librerías requeridas
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
import warnings

tf.enable_v2_behavior()

plt.style.use("ggplot")
warnings.filterwarnings('ignore')

In [3]:
# Los datos
J = 8 #Número de escuelas
y = np.array([28.,  8., -3.,  7., -1.,  1., 18., 12.], dtype=np.float32) #Estimaciones de los efectos del tratamiento
sigma = np.array([15., 10., 16., 11.,  9., 11., 10., 18.], dtype=np.float32) #Estimacion de la desviacion de los efectos del tratamiento
schools = np.array(['Choate', 'Deerfield', 'Phillips Andover', 'Phillips Exeter',
                    'Hotchkiss', 'Lawrenceville', "St. Paul's", 'Mt. Hermon'])

schools_dat = {'J': J,
               'y': y,
               'sigma': sigma}

## <span style="color:#4361EE">Implementación del modelo estadístico en TFP</span>


In [4]:
model = tfd.JointDistributionSequential([
  tfd.Normal(loc=0., scale=tf.math.sqrt(5.), name="mu"),  # `mu` 
  tfd.HalfCauchy(loc=0., scale=5., name="tau"),  # `Tau``
  tfd.Independent(tfd.Normal(loc=tf.zeros(J),
                             scale=tf.ones(J),
                             name="eta"),  # eta_i. Recordemos que eta_i es un vector
                  reinterpreted_batch_ndims=1),
  lambda eta, tau, mu: (
      tfd.Independent(tfd.Normal(loc=(mu[..., tf.newaxis] +
                                      tau[..., tf.newaxis]* 
                                      eta),  # `theta` 
                                 scale=sigma),
                      name="y",  # `y` 
                      reinterpreted_batch_ndims=1))
])

def target_log_prob_fn(mu, tau, eta):
  """Densidad objetivo no normalizada como función de estados."""
  return model.log_prob((
      mu, tau, eta, sigma))


En este código los parámetros que serán muestreados son $\mu$ (*media global*), $\tau$ (*desviación estándar*), y los $\eta_i$ (*eta*) que son los efectos aleatorios en el modelo. Las medias $\theta_i$ de las escuelas son calculados. Es decir, son funcionales de los parámetros.

Hemos asumido las distribuciones a priori para $\mu\sim \mathcal{N}(0,5)$,  $\tau\sim \mathcal{Cauchy}(0,5)1_{\tau>0.0}$ y $\eta_i \sim \mathcal{N}(0,1)$.

## <span style="color:#4361EE">Compilación del modelo</span>


In [5]:
num_results = 500
num_burnin_steps = 500

# Mejorar el rendimiento rastreando el muestreador con `tf.function`
# y compilarlo usando XLA.
# autograph transforma un subconjunto de código de Python a TensorFlow
#jit_compile compila la función usando XLA
@tf.function(autograph=False, jit_compile=True)
def do_sampling():
    # Muestreador que usa cadenas de Markov
  return tfp.mcmc.sample_chain(
      num_results=num_results,
      num_burnin_steps=num_burnin_steps,
      # Representación de nuestra previa
      current_state=[
          tf.zeros([], name='mu_0'),
          tf.zeros([], name='tau_0'),
          tf.ones([J], name='eta_0'),
      ],
      kernel=tfp.mcmc.HamiltonianMonteCarlo(
          # Devuelve la log-densidad de nuestra previa
          target_log_prob_fn=target_log_prob_fn,
          step_size=0.4,
          num_leapfrog_steps=3))


In [6]:
states, kernel_results = do_sampling()
mu, tau, eta = states

### <span style="color:#4CC9F0">Distribución Predictiva Posterior</span>


In [7]:
sample_shape = [5000]

_, _, _, predictive_effects = model.sample(
    value=(tf.broadcast_to(np.mean(mu, 0), sample_shape),
           tf.broadcast_to(np.mean(tau, 0), sample_shape),
           tf.broadcast_to(np.mean(eta, 0),
                           sample_shape + [J]),
           None))


## <span style="color:#4361EE">Inferencia. InferenceData(ArviZ)</span>


Para seguir esta parte del tutorial localmente por favor instale ArviZ.

Desde la linea de comandos: 

*pip install arviz*

o mejor use

*conda install -c conda-forge arviz*

[InferenceData](https://arviz-devs.github.io/arviz/notebooks/InferenceDataCookbook.html) es la principal estructura de datos de ArviZ, la herramienta recomendada para nuestros análisis con TFP. 

In [8]:
# Importamos las librerías requeridas.

import arviz as az
import numpy as np
import pandas as pd
import xarray

# definimos estilos para el fondo de los gráficos y para mostrar la información
az.style.use('arviz-darkgrid')
xarray.set_options(display_style="html");

In [12]:
tfp_data = az.from_tfp (
    posterior = kernel_results,
    posterior_predictive_samples=500,
    coords={'school': schools }, 
    dims={
        "theta": ["school"],
        'y': ['school'],
        'eta': ['school'],
    }
)

ModuleNotFoundError: No module named 'tensorflow_probability.python.edward2'

In [ ]:
tfp_data

In [ ]:
tfp_data.posterior

In [ ]:
tfp_data.posterior.all

In [ ]:
tfp_data.posterior.mu

In [ ]:
val = tfp_data.posterior.mu[0,300]
print(val)
print('np.array(val) = ' ,np.array(val))


In [ ]:
tfp_data.sample_stats


In [ ]:
tfp_data.log_likelihood


In [ ]:
tfp_data.posterior_predictive


In [ ]:
tfp_data.observed_data


In [ ]:
az.plot_density(tfp_data, var_names=['mu', 'tau']);


In [ ]:
az.plot_density?


In [ ]:
az.plot_density(tfp_data);


In [ ]:
az.plot_autocorr(tfp_data, var_names=("tau","mu"))


In [ ]:
az.plot_autocorr?


In [ ]:
az.plot_autocorr(tfp_data,max_lag=40,var_names=("tau","mu"))


In [ ]:
az.plot_forest(tfp_data, var_names=("tau","mu","eta"))

In [ ]:
az.plot_forest?


In [ ]:
az.plot_forest(tfp_data, var_names=("tau","mu", "eta"),r_hat=True)


In [ ]:
az.plot_trace(tfp_data, var_names=("tau","mu"))


In [ ]:
az.plot_trace?


In [ ]:
az.plot_mcse(tfp_data)

In [ ]:
az.plot_mcse?


In [ ]:
az.plot_pair(tfp_data, coords={"school": ['Choate', 'Deerfield', 'Phillips Andover']}, divergences=True);


In [ ]:
az.geweke(tfp_data.posterior.mu[0,:])

## <span style="color:#4361EE">Referencias</span>

- https://arviz-devs.github.io/arviz/getting_started/CreatingInferenceData.html#from-pyro